In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
trn_path= '../input/flower_data/flower_data/train'
vld_path= '../input/flower_data/flower_data/valid'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(trn_path, transform=train_transforms)
test_data =  datasets.ImageFolder(vld_path, transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [3]:
model = models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /tmp/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:03<00:00, 75091438.13it/s]


In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 500)),
                          ('relu', nn.Dropout(0.2)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.fc = classifier

In [5]:
from torch.optim import lr_scheduler
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.3)
model.to(device)
criterion = criterion.cuda()

In [6]:
import numpy as np
# number of epochs to train the model
n_epochs = 14
train_on_gpu = True
valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    exp_lr_scheduler.step()
    for batch_idx, (data, target) in enumerate(trainloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(testloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(testloader.dataset)
    accuracy  = accuracy/len(testloader)
        
    # print training/validation statistics 
    print('Epoch: {} \tTrain_Loss: {:.6f} \tVali_Loss: {:.6f} \tValid_accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Train_Loss: 2.801837 	Vali_Loss: 1.056292 	Valid_accuracy: 0.696731
Validation loss decreased (inf --> 1.056292).  Saving model ...
Epoch: 2 	Train_Loss: 1.066743 	Vali_Loss: 0.613693 	Valid_accuracy: 0.838510
Validation loss decreased (1.056292 --> 0.613693).  Saving model ...
Epoch: 3 	Train_Loss: 0.754732 	Vali_Loss: 0.436781 	Valid_accuracy: 0.884519
Validation loss decreased (0.613693 --> 0.436781).  Saving model ...
Epoch: 4 	Train_Loss: 0.536468 	Vali_Loss: 0.322721 	Valid_accuracy: 0.920192
Validation loss decreased (0.436781 --> 0.322721).  Saving model ...
Epoch: 5 	Train_Loss: 0.493612 	Vali_Loss: 0.282002 	Valid_accuracy: 0.920385
Validation loss decreased (0.322721 --> 0.282002).  Saving model ...
Epoch: 6 	Train_Loss: 0.467043 	Vali_Loss: 0.290291 	Valid_accuracy: 0.930673
Epoch: 7 	Train_Loss: 0.430702 	Vali_Loss: 0.261588 	Valid_accuracy: 0.932212
Validation loss decreased (0.282002 --> 0.261588).  Saving model ...
Epoch: 8 	Train_Loss: 0.405851 	Vali_Loss: 0.

In [7]:
for param in model.parameters():
    param.requires_grad = True


optimizer = optim.Adam(model.parameters(), lr=1e-5)#optimizer = torch.optim.SGD([{'params': model.parameters()}], lr=1e-5)



model.to(device)
criterion = criterion.cuda()

In [8]:
trainloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=8)

In [9]:
import numpy as np
# number of epochs to train the model
n_epochs = 10
train_on_gpu = True

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    exp_lr_scheduler.step()
    for batch_idx, (data, target) in enumerate(trainloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(testloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(testloader.dataset)
    accuracy  = accuracy/len(testloader)
        
    # print training/validation statistics 
    print('Epoch: {} \tTrain_Loss: {:.6f} \tVali_Loss: {:.6f} \tValid_accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Train_Loss: 0.555060 	Vali_Loss: 0.192836 	Valid_accuracy: 0.951456
Validation loss decreased (0.244172 --> 0.192836).  Saving model ...
Epoch: 2 	Train_Loss: 0.431984 	Vali_Loss: 0.156842 	Valid_accuracy: 0.968447
Validation loss decreased (0.192836 --> 0.156842).  Saving model ...
Epoch: 3 	Train_Loss: 0.356984 	Vali_Loss: 0.138099 	Valid_accuracy: 0.968447
Validation loss decreased (0.156842 --> 0.138099).  Saving model ...
Epoch: 4 	Train_Loss: 0.308121 	Vali_Loss: 0.118345 	Valid_accuracy: 0.976942
Validation loss decreased (0.138099 --> 0.118345).  Saving model ...
Epoch: 5 	Train_Loss: 0.273416 	Vali_Loss: 0.122006 	Valid_accuracy: 0.972087
Epoch: 6 	Train_Loss: 0.266815 	Vali_Loss: 0.099462 	Valid_accuracy: 0.974515
Validation loss decreased (0.118345 --> 0.099462).  Saving model ...
Epoch: 7 	Train_Loss: 0.235720 	Vali_Loss: 0.098314 	Valid_accuracy: 0.974515
Validation loss decreased (0.099462 --> 0.098314).  Saving model ...
Epoch: 8 	Train_Loss: 0.198014 	Vali_Los

In [10]:
from IPython.display import FileLink
FileLink('model_augmented.pt')

/kaggle/working/model_augmented.pt

In [12]:
model.load_state_dict( torch.load('model_augmented.pt'))

In [13]:
import numpy as np
# number of epochs to train the model
n_epochs = 10
train_on_gpu = True

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    accuracy = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    exp_lr_scheduler.step()
    for batch_idx, (data, target) in enumerate(trainloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(testloader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.dataset)
    valid_loss = valid_loss/len(testloader.dataset)
    accuracy  = accuracy/len(testloader)
        
    # print training/validation statistics 
    print('Epoch: {} \tTrain_Loss: {:.6f} \tVali_Loss: {:.6f} \tValid_accuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Train_Loss: 0.175016 	Vali_Loss: 0.079022 	Valid_accuracy: 0.985437
Validation loss decreased (0.081830 --> 0.079022).  Saving model ...
Epoch: 2 	Train_Loss: 0.174524 	Vali_Loss: 0.098400 	Valid_accuracy: 0.976942
Epoch: 3 	Train_Loss: 0.164452 	Vali_Loss: 0.078287 	Valid_accuracy: 0.983010
Validation loss decreased (0.079022 --> 0.078287).  Saving model ...
Epoch: 4 	Train_Loss: 0.165037 	Vali_Loss: 0.078096 	Valid_accuracy: 0.983010
Validation loss decreased (0.078287 --> 0.078096).  Saving model ...
Epoch: 5 	Train_Loss: 0.158797 	Vali_Loss: 0.073990 	Valid_accuracy: 0.985437
Validation loss decreased (0.078096 --> 0.073990).  Saving model ...
Epoch: 6 	Train_Loss: 0.152567 	Vali_Loss: 0.071091 	Valid_accuracy: 0.989078
Validation loss decreased (0.073990 --> 0.071091).  Saving model ...
Epoch: 7 	Train_Loss: 0.156323 	Vali_Loss: 0.092374 	Valid_accuracy: 0.979369
Epoch: 8 	Train_Loss: 0.134140 	Vali_Loss: 0.072307 	Valid_accuracy: 0.989078
Epoch: 9 	Train_Loss: 0.137717 

In [18]:
FileLink('model_augmented.pt')

/kaggle/working/model_augmented.pt